In [1]:
import os
import json
import numpy as np
import jieba
from collections import Counter

In [2]:
infile = './兔司机与候选人首次沟通录音.json'

In [3]:
dialog_dict = []

In [4]:
with open(infile, 'r', encoding='utf8') as f:
    dialog_dict = json.load(f)

In [5]:
print('通话数量：{}'.format(len(dialog_dict)))

通话数量：49


In [6]:
dialog_nums_dict = {}
for key, val in dialog_dict.items():
    dialog_nums_dict[key] = len(json.loads(val['data']))

In [7]:
total_cnt = 0
val_list = []
for key, val in dialog_nums_dict.items():
    total_cnt += int(val)
    val_list.append(int(val))
print('文件个数：{}'.format(len(dialog_dict)))
print('平均每个对话的句子数:{}'.format(int(total_cnt/len(dialog_dict))))
print('最长句子数：{}'.format(max(val_list)))
print('最短句子数：{}'.format(min(val_list)))

文件个数：49
平均每个对话的句子数:78
最长句子数：292
最短句子数：11


In [8]:
# with open('./dialog.txt', 'w', encoding='utf8') as f:
#     for key, val in dialog_dict.items():
#         f.write('====> {}'.format(key))
#         f.write('\n')
#         for it in json.loads(val['data']):
#             sent = '\tspeaker {}:\t{}'.format(it['speaker'], it['onebest'])
#             f.write(sent)
#             f.write('\n\n')
#         f.write('\n\n\n\n')

In [9]:
words_list = []
for key,val in dialog_dict.items():
    for it in json.loads(val['data']):
        sent = it['onebest']
        ss = list(jieba.cut(sent))
        words_list += ss

Building prefix dict from /Users/higgs/anaconda2/envs/py36/lib/python3.6/site-packages/jieba/dict.txt ...
Dumping model to file cache /var/folders/9s/drr9_t055tz9y75f7gx80s980000gp/T/jieba.cache
Loading model cost 1.7890970706939697 seconds.
Prefix dict has been built succesfully.


In [10]:
words_prequency = Counter(words_list)

In [11]:
words_prequency = sorted(words_prequency.items(), key=lambda x: x[1], reverse=True)

In [12]:
words_prequency

[('，', 4363),
 ('的', 2637),
 ('。', 2007),
 ('是', 1964),
 ('嗯', 1423),
 ('我', 1413),
 ('对', 895),
 ('您', 874),
 ('？', 847),
 ('然后', 845),
 ('就是', 815),
 ('的话', 693),
 ('这个', 668),
 ('啊', 659),
 ('这边', 600),
 ('有', 579),
 ('你', 579),
 ('在', 514),
 ('一个', 505),
 ('那', 496),
 ('也', 449),
 ('吧', 445),
 ('吗', 431),
 ('就', 399),
 ('了', 390),
 ('好', 380),
 ('公司', 370),
 ('那个', 352),
 ('说', 350),
 ('一下', 332),
 ('因为', 317),
 ('噢', 308),
 ('做', 297),
 ('这', 297),
 ('一些', 292),
 ('！', 291),
 ('他们', 282),
 ('不', 281),
 ('可以', 273),
 ('都', 265),
 ('现在', 265),
 ('可能', 262),
 ('他', 255),
 ('呢', 248),
 ('看', 234),
 ('唉', 212),
 ('嘛', 209),
 ('我们', 202),
 ('给', 199),
 ('去', 192),
 ('什么', 188),
 ('工作', 187),
 ('跟', 182),
 ('还是', 178),
 ('到', 176),
 ('目前', 174),
 ('了解', 170),
 ('比较', 167),
 ('没有', 166),
 ('这样', 165),
 ('会', 159),
 ('就是说', 156),
 ('所以', 137),
 ('微信', 134),
 ('其实', 134),
 ('行', 129),
 ('一块', 129),
 ('不是', 128),
 ('主要', 125),
 ('要', 125),
 ('想', 124),
 ('还有', 121),
 ('之前', 118),
 ('上海', 116

In [13]:
# 最简单的词频统计，得不到任何有意义的信息
# 下一步将每句话用bert进行语义编码，然后进行聚类，看能得到什么结论？

In [14]:
from bert_serving.client import BertClient

In [15]:
ikey = 0
sent_list = []
for key, val in dialog_dict.items():
    for it in json.loads(val['data']):
        sent_list.append(it['onebest'])

In [16]:
bc = BertClient()

In [17]:
sent_arr = bc.encode(sent_list)

In [18]:
sent_arr.shape

(3868, 768)

In [19]:
from sklearn.cluster import KMeans

In [20]:
sent_cls=KMeans(n_clusters=30).fit_predict(sent_arr)

In [21]:
sent_cls[:30]

array([ 6, 22, 11, 21, 18, 25, 25, 21, 16, 15, 21, 23,  2, 29,  9, 29,  3,
       25,  4, 19,  3,  5, 21, 11, 25, 18,  3,  8, 18,  5], dtype=int32)

In [22]:
label2sent = {}
for sent, label in zip(sent_list, sent_cls):
    if label not in label2sent:
        label2sent[label] = [sent]
    else:
        label2sent[label].append(sent)

In [23]:
label2sent[4]

['对，是要我这边先推过去，然后那边hr跟技术部门先看一下您的简历，如果觉得合适的话，我这边跟您约面对。',
 '嗯目前来说我给你简单介绍一下吧好吧，唉您目前对徐汇区这块的话上班距离方便吗？',
 '对，我理解我就是说像您这样的一些经验综合起来的话会更好。',
 '嗯我这边的话就是有一个职位想给您推荐一下，公司呢是叫这个高端科技，',
 '啊就是我们这就是现在这个招聘是计划是年年后入职还是什么？',
 '对对，所以说他们这样的人群他们也是非常喜欢的，所以嗯我还想给您推荐一下，然后稍后帮您推荐一下，然后这边的话就是说需要您提供一下这种求职照片，您看可以吗？',
 '嗯因为我之前看到您的简历那个照片上看起来那个。',
 '唉好勒好勒，那我稍后添加一下，然后嗯还不了解，就是说我看您在目前这家公司是只做了九个月，您之前的话都是',
 '然后还有一点就是想问一下您目前的话在这家公司是是九个月吗？',
 '嗯好的好的，唉我看您其实嗯那您在这个公司的话，薪资大概是怎样的呢。',
 '嗯喂刚刚不好意思，那个信号不太好，就是两个字，然后断一下，两个断一下，就是我们听听嗯。',
 '嗯那您这边就是说嗯期望薪资是怎样的？',
 '啊你好，邓先生是这样的，我这边呢有一个职位，就是说它的名字呢叫人性啊科技，我不知道你有没有听过？',
 '噢大于那您现在我看到您现在的嗯待遇，你写的是30K然后近16个月对吗？',
 '嗯您现在就是除了就主要这个待遇问题对吗？',
 '嗯我问一下您现在是住在北京哪个区？啊您对地点这一方面的话会不会介意啊。',
 '呃就北京这边的话是在海淀区，但是呢这个其实也不是坐班，因为您知道的这一块就因为正常也不会坐班这样子，这样，我我把他们的在海地',
 '嗯好好，对对对噢对，我帮看看一下，吧我发给你看一下吧他们公司那个信息好不好，包括他的那个职位的一个界定，您先了解一下，因为主要是信息安全与智慧城市这一块的，就是主要是这两块，他们家。',
 '噢对，嗯然后您是在北京哪个区？啊我这边有一个机会想给你推荐一下，是。',
 '你好你好，是这样子的，我看到您这边的话在网站上放简历，然后咱们目前是在看工作是吧？',
 '嗯嗯您目前的话咱们这边还还是在职的是吧？',
 '嗯嗯是这样子的，因为我看到您这边的话，这边是有一个您公司的话应该是嗯中建材信息股份有限公司是吧？',
 '对对，因

In [24]:
# 总结下所有的问句
ask_sent_list = []
for sent in sent_list:
    if '？' in sent:
        ask_sent_list.append(sent)

In [25]:
len(ask_sent_list)

787

In [26]:
ask_sent_list[:10]

['喂您好唉您好，请问是朱先生吗？',
 '哦我这边是希格斯的猎头顾问，想问一下您目前有在看新工作吗？',
 '噢已经找到新工作了是吧？',
 '你你有你有什么推荐吗？还没有去！',
 '噢我这边是看到您期望地点在广州，嘛然后帮您看了几家广州的机会，一个叫超联软件，您之前有过接触吗？',
 '公司的嗯他们公司的地点在哪里？',
 '噢或者哦是让你先去推过去，对吧？',
 '我先了解一下您的一些信息，吧好吧？',
 '您现在是离职状态是吗？',
 '噢是是一就是18年10月份就离职了吗？']

In [27]:
candidate_ask_list = []
hunter_ask_list = []
for key, val in dialog_dict.items():
    ask_dict = {}
    for it in json.loads(val['data']):
        if '？' not in it['onebest']:
            continue
        speaker = it['speaker']
        if speaker in ask_dict:
            ask_dict[speaker].append(it['onebest'])
        else:
            ask_dict[speaker] = [it['onebest']]
    ask_l = sorted(ask_dict.items(), key=lambda x:len(x[1]))
    if len(ask_l) == 0:
        continue
    elif len(ask_l) == 1:
        hunter_ask_list.append(ask_l[0][1])
    elif len(ask_l) == 2:
        try:
            if len(ask_l[0]) > 1:
#                 candidate_ask_list.append(ask_l[0][1])
                candidate_ask_list += ask_l[0][1]
            if len(ask_l[1]) > 1:
#                 hunter_ask_list.append(ask_l[1][1])
                hunter_ask_list += ask_l[1][1]
        except:
            print(ask_l)
        

In [28]:
len(candidate_ask_list)

156

In [29]:
candidate_ask_list

['你你有你有什么推荐吗？还没有去！',
 '公司的嗯他们公司的地点在哪里？',
 '噢或者哦是让你先去推过去，对吧？',
 '噢不是不是唉这样？吧我我这里说不太方便，要是。',
 '开在什么？',
 '然后就最近微盟吗微盟您知道吗？',
 '你好，陈是曹继元先生吗？',
 '嗯您现在方便沟通吗？我可以给您简单介绍一下，或者如果您不方便的话，可以加微信发给您。',
 '这个地址要离你远不远？',
 '嗯嗯好的好的好的，那我加您微信，您这个手机号就是您微信号吗？',
 'OK吗？我。',
 '然后工作内容就是主要是你刚才说的这些是吧？',
 '啊就是我们这就是现在这个招聘是计划是年年后入职还是什么？',
 '什么？',
 '啊最新版的是吧？嗯',
 '然后我发给你可以吗？',
 '人性啊是吧？',
 '做哪一个产品？',
 '已经没有人是吧？嗯。',
 '到底怎么样？它这里。',
 '待遇怎么样？',
 '什么什么科技？',
 '不是不是不是政策政策？',
 '啊这个是不是几个月的通知期了？是其实在下半年的时候就已经。',
 '对，那我们是合作伙伴关系吗？',
 '万知道吧？',
 '什么时候来大概？',
 '过期没关系，啊这个我我我我不会去的了，因为因为这东西序的也也是没用，你在招投标的时候也用不上了，对吧？不过是说，',
 '的在哪里？',
 '做那个社交平台是吧？',
 '然后说他们公司叫什么名字？',
 '阿里是叫什么名字？',
 '当当信息科技是吧？',
 '你刚才说什么？没听清办公厅原怎么的！',
 '那现在你现在的一个就是说每月的薪资是吧？',
 '唉对哪位你好？',
 '是是这样，我是想问一下，就是咱们这个岗位是属于纯文池的是吗？',
 '那在什么位置？',
 '咱们这个岗位的这个薪资待遇大概是怎样？',
 '到20是吗？嗯。',
 '喂你好，周先生是吗？',
 '你现在方便说话吗？',
 '就是给你发了offer，然后您您没有考虑那边是吗？',
 '嗯然然后当时我们是出于什么考虑呢？话，最后没有选择改变。',
 '当时他跟你是说签签的是什么合同？是正式合同吗？就是远东跟远东这边之前还是说？',
 '然后我们这边建设目前这边签的全是正式合同，然后我们的薪资架构，然后跟您之前了解的可能也不太一样，如果说我们这边您能顺利拿到offer的话，我们是24星最低

In [30]:
hunter_ask_list

['喂您好唉您好，请问是朱先生吗？',
 '哦我这边是希格斯的猎头顾问，想问一下您目前有在看新工作吗？',
 '噢已经找到新工作了是吧？',
 '噢我这边是看到您期望地点在广州，嘛然后帮您看了几家广州的机会，一个叫超联软件，您之前有过接触吗？',
 '我先了解一下您的一些信息，吧好吧？',
 '您现在是离职状态是吗？',
 '噢是是一就是18年10月份就离职了吗？',
 '噢我加一下您微信吧是这个手机号吗？',
 ['唉喂你好，是杨旭杨小姐吗？唉你好，是杨先生杨小姐吗？',
  '嗯我爱我这边是智客网络的一个招聘顾问，吧我这边有个叫留学规划师这块的，想与您沟通一下，您现在还有在开学机会吗？',
  '嗯目前来说我给你简单介绍一下吧好吧，唉您目前对徐汇区这块的话上班距离方便吗？',
  '嗯他全国是有800多人的一个规模，嘛所以说目前也在扩张阶段，嗯目前来说他招的一个留学顾问就可能说，比如说以前有个留美的一个留学规划这块的一个经验，嘛那你以前是负责哪块业务的？',
  '嗯好的，我12点半左右打给你，然后到时候你接听一下好吧？嗯好的嗯好，唉我微信加进来可以先呃先加一下，然后我们可以微信简单沟通一下。'],
 '唉喂你好，请问是陈先生吗？',
 '唉你好，我这边是那个上海希格斯猎头顾问，然后想问一下您这会方便吗？',
 '啊是在苏州这边微盟吗？',
 '好的，那就是像现在的话还考虑其他一些工作机会吗？',
 '教外国人说汉语是吧？',
 '嗯大概是在哪个区域？啊我的这个地图不太熟。',
 '嗯还有什么信息吗？',
 '他们想做什么产品？',
 '教汉语吗？你为什么嗯',
 '干嘛去做来是用来教学呢还是用来推广？',
 '对吧？这些。',
 '好的，他们又他们老是自己培养的，还是说？',
 ['唉喂您好，请问您这边是梁倩吗？什么？',
  '嗯这边是2000吗？',
  '这块确定了吗？',
  '噢那先来看一下有新的机会吗？就是。',
  '是之前嗯推荐过过你，还是说之前已经接触过了？',
  '了解，不是对他们工作这一块没有太大兴趣吗？还是什么舞！',
  '噢工作量这一块，那你现在因为我看你之前是亚马逊这一块，那你现在在工作还是做亚马逊这一块吗？',
  '微信聊是吧？好，呀你的手机。',
  '噢那我加你微信，对，你主要把你几个点告诉我一下，我然后帮您匹配一下好吗